# Homework: Vaccine Distribution

###  Bus 36109 "Advanced Decision Modeling with Python", Don Eisenstein
Don Eisenstein &copy; Copyright 2022, University of Chicago 



A vaccine is being produced at different manufacturing Plants to be distributed across the United States to various Hospitals. Our task is to determine the quantities each Plant should ship to each Hospital.

There are 6 manufacturing facilities, each with a limited weekly capacity (in cases) of vaccine doses. The location of each facility is represented as `(x, y)` map coordinates. 

| Facility | Location   | Capacity |
| -------- | :----------: | ------: |
| Plant 1  | (123, 210) | 200    |
| Plant 2  | (40, 71)   | 50    |
| Plant 3  | (21, 185)  | 150    |
| Plant 4  | (129, 57)  | 40    |
| Plant 5  | (300, 12)  | 100    |
| Plant 6  | (281, 190) | 100    |

There are ten hospital networks in need of vaccines. Their weekly quantities needed (in cases) and `(x, y)` map coordinates are shown below.

| Facility | Location   | Demand |
| -------- | :----------: | ------: |
| Hospital 1  | (41, 21) | 50    |
| Hospital 2  | (129, 44)   | 75    |
| Hospital 3  | (61, 210)  | 121    |
| Hospital 4  | (78, 47)  | 231    |
| Hospital 5  | (12, 90)  | 147    |
| Hospital 6  | (70, 250) | 190    |
| Hospital 7  | (63, 194)   | 100    |
| Hospital 8  | (199, 28)  | 151    |
| Hospital 9  | (351, 7)  | 49    |
| Hospital 10  | (222, 163)  | 172    |

Assumption:  You can assume, as this data indicates, that the total Plant capacity is no greater than the Hospital demand.

We estimate our cost of transportation by considering the *Rectilinear* distance between each Plant and Hospital.  Rectilinear travel restricts movement along the horizontal (x) and vertical (y) axes. It is commonly used to approximate travel along a road grid. 

The transportation cost is $1 per vaccine case traveling one unit of our map distance. The rectilinear distance between two points `(x_1, y_1)` and `(x_2, y_2)` is `abs(x_2 - x_1) + abs(y_2 - y_1)`.   The Python `abs` function returns the absolute value of a number.

For example, the distance between `(0, 4)` and `(2, 1)` is `abs(2 - 0) + abs(1 - 4) = 5`, and transporting `100` vaccine cases would therefore cost `100 cases * 5 distance units * $1/(case-distance) = $500`. 

Each manufacturing plant seeks to send as many cases as possible in order to meet hospital demand. Because vaccines are extremely valuable, hospitals should not receive any additional cases exceeding their immediate requirements.  Each manufacturing plant can ship vaccines to multiple hospitals, and each hospital can receive vaccines from multiple manufacturing plants. You can assume that the number of vaccine cases sent and received is always an integer number.

Your model should answer the following question:

**What is the optimal allocation of vaccine cases to hospitals that minimizing total transportation cost, without exceeding the requirements of any hospital?   In other words, how many cases of vaccines should each manufacturing plant send to each hospital?** 

Follow the notebook to walk you through the solution in parts.  Insert your answer to each part into the notebook below the question for each part.  

In [1]:
# === SETUP ===
import pulp
import os
from pprint import pprint

# Portable solver setup, to allow work locally (ARM64 architecture) as well as in JupyterHub
from pulp import COIN_CMD
if os.path.exists('/opt/homebrew/opt/cbc/bin/cbc'):
    solver = COIN_CMD(path='/opt/homebrew/opt/cbc/bin/cbc', msg=0)
else:
    solver = pulp.PULP_CBC_CMD(msg=0)

# Your Solution

Insert your answer to each part into this notebook

**1. In broad terms, what are the variables, objective and constraints of this problem? You don't need to list the entire formulation. Just describe the structure/characteristics of your model.**

- Variables: $case_{ij}$, number of vacine cases sent from Plant $i$ to Hospital $j$ (60 $ij$ pairs).

- Objective: minimize the sum of total rectiliniar distance travelled by each of case.
$$\min \sum_{i=1}^{6} \sum_{j=1}^{10} (\text{rectiliniar distance}_{ij} \times \text{case}_{ij})$$


- Constraint set 1: for each plant $(i = 1, ..., 6)$, the sum of all cases traveling to hospitals $(j = 1, ..., 10)$ from it must equal its capacity.
$$\sum_{j=1}^{10}(\text{case}_{ij}) = \text{capacity}_{i} \quad i = 1,...,6$$


- Constraint set 2: for each hospital $(j = 1, ..., 10)$, the sum of all cases traveling from plants $(i = 1, ..., 6)$ must not exceed its demand.
$$\sum_{i=1}^{6}(\text{case}_{ij}) \le \text{demand}_{j} \quad j = 1,...,10$$

- Constraint set 3: each $\text{case}_{ij}$ is non-negative integers.


Now, translate your model into Python so you can solve it.

**2.  Set up a Python List called `plants` to store the information about the plants.   
Each element of the List should be a Dictionary that will have Keys `name`, `capacity`, `x` and `y` with associated data from the table above.**


In [2]:
plants = [
    {'name': 'Plant 1', 'capacity': 200, 'x': 123, 'y': 210}, 
    {'name': 'Plant 2', 'capacity': 50, 'x': 40, 'y': 71},
    {'name': 'Plant 3', 'capacity': 150, 'x': 21, 'y': 185},
    {'name': 'Plant 4', 'capacity': 40, 'x': 129, 'y': 57},
    {'name': 'Plant 5', 'capacity': 100, 'x': 300, 'y': 12},
    {'name': 'Plant 6', 'capacity': 100, 'x': 281, 'y': 190}
]

**3.  Now do a similar thing for hospitals, where each element of a `hospitals` List will be a Dictionary with keys `name`, `demand`, `x` and `y` with associated data fromt the table above.**

In [3]:
hospitals = [ 
    {'name': 'Hospital 1', 'demand': 50, 'x': 41, 'y': 21},
    {'name': 'Hospital 2', 'demand': 75, 'x': 129, 'y': 44},
    {'name': 'Hospital 3', 'demand': 121, 'x': 61, 'y': 210},
    {'name': 'Hospital 4', 'demand': 231, 'x': 78, 'y': 47},
    {'name': 'Hospital 5', 'demand': 147, 'x': 12, 'y': 90},
    {'name': 'Hospital 6', 'demand': 190, 'x': 70, 'y': 250},
    {'name': 'Hospital 7', 'demand': 100, 'x': 63, 'y': 194},
    {'name': 'Hospital 8', 'demand': 151, 'x': 199, 'y': 28},
    {'name': 'Hospital 9', 'demand': 49, 'x': 351, 'y': 7},
    {'name': 'Hospital 10', 'demand': 172, 'x': 222, 'y': 163}
]

**4. Create a PuLP LpProblem object and store it in the variable `model`.** 

In [4]:
model = pulp.LpProblem('Vaccine', pulp.LpMinimize)

**5. Create a Dictionary `lp_variables` and store in it each PuLP variable.  Each key in the `lp_variables` Dictionary should be a Tuple listing the plant name and hospital name** 

In [5]:
# Start with an empty dictionary of cases shipped
shipments = {}

# Iterate over plant-hospital pairs (i-j) and create LpVariables for each
# Key: tuple(plant_name, hospital_name)
# Value: LpVariable representing the number of cases shipped on that route
for plant in plants:
    for hospital in hospitals:
        # Define pair
        pair = (plant['name'],hospital['name'])
        # Create PuLP variable for the pair and put in the dictionary
        var_name = f"Cases_{plant['name']}_{hospital['name']}"
        shipments[pair] = pulp.LpVariable(var_name, lowBound = 0, cat='Integer')

**6. Add your objective function to your `model`**

In [6]:
# Set up total cost of transportation as objective function
cost = 0.0

# Iterate over plant-hospital pairs (i-j) and calculate distance between them
# Sum cost across all routes: distance × cases shipped × $1 per case-unit
for plant in plants:
    for hospital in hospitals:
        pair = (plant['name'],hospital['name'])
        distance = abs(plant['x']-hospital['x']) + abs(plant['y']-hospital['y'])
        cost += distance * shipments[pair] * 1

# Add cost function into the model as an objective
model += cost
print("Objective Function Formulation:", model.objective)

Objective Function Formulation: 271*Cases_Plant_1_Hospital_1 + 146*Cases_Plant_1_Hospital_10 + 172*Cases_Plant_1_Hospital_2 + 62*Cases_Plant_1_Hospital_3 + 208*Cases_Plant_1_Hospital_4 + 231*Cases_Plant_1_Hospital_5 + 93*Cases_Plant_1_Hospital_6 + 76*Cases_Plant_1_Hospital_7 + 258*Cases_Plant_1_Hospital_8 + 431*Cases_Plant_1_Hospital_9 + 51*Cases_Plant_2_Hospital_1 + 274*Cases_Plant_2_Hospital_10 + 116*Cases_Plant_2_Hospital_2 + 160*Cases_Plant_2_Hospital_3 + 62*Cases_Plant_2_Hospital_4 + 47*Cases_Plant_2_Hospital_5 + 209*Cases_Plant_2_Hospital_6 + 146*Cases_Plant_2_Hospital_7 + 202*Cases_Plant_2_Hospital_8 + 375*Cases_Plant_2_Hospital_9 + 184*Cases_Plant_3_Hospital_1 + 223*Cases_Plant_3_Hospital_10 + 249*Cases_Plant_3_Hospital_2 + 65*Cases_Plant_3_Hospital_3 + 195*Cases_Plant_3_Hospital_4 + 104*Cases_Plant_3_Hospital_5 + 114*Cases_Plant_3_Hospital_6 + 51*Cases_Plant_3_Hospital_7 + 335*Cases_Plant_3_Hospital_8 + 508*Cases_Plant_3_Hospital_9 + 124*Cases_Plant_4_Hospital_1 + 199*Cases_Pl

**7. Add the constraints to your `model`**

In [7]:
# Constraint set 1: Each plant ships exactly its capacity
for plant in plants:
    # Sum all shipments from plant i to all hospitals
    total_shipped = 0.0
    for hospital in hospitals:
        total_shipped += shipments[(plant['name'], hospital['name'])]
    model += total_shipped == plant['capacity'], f'Capacity_{plant['name']}'

# Constraint set 2: Each hospital receives at most its demand
for hospital in hospitals:
    # Sum all shipments from all plants to hospitals j
    total_received = 0.0
    for plant in plants:
        total_received += shipments[(plant['name'], hospital['name'])]
    model += total_received <= hospital['demand'], f'Demand_{hospital['name']}'

**8. Display your model with `print(model)`, check that all is OK**

In [8]:
print(model)

Vaccine:
MINIMIZE
271*Cases_Plant_1_Hospital_1 + 146*Cases_Plant_1_Hospital_10 + 172*Cases_Plant_1_Hospital_2 + 62*Cases_Plant_1_Hospital_3 + 208*Cases_Plant_1_Hospital_4 + 231*Cases_Plant_1_Hospital_5 + 93*Cases_Plant_1_Hospital_6 + 76*Cases_Plant_1_Hospital_7 + 258*Cases_Plant_1_Hospital_8 + 431*Cases_Plant_1_Hospital_9 + 51*Cases_Plant_2_Hospital_1 + 274*Cases_Plant_2_Hospital_10 + 116*Cases_Plant_2_Hospital_2 + 160*Cases_Plant_2_Hospital_3 + 62*Cases_Plant_2_Hospital_4 + 47*Cases_Plant_2_Hospital_5 + 209*Cases_Plant_2_Hospital_6 + 146*Cases_Plant_2_Hospital_7 + 202*Cases_Plant_2_Hospital_8 + 375*Cases_Plant_2_Hospital_9 + 184*Cases_Plant_3_Hospital_1 + 223*Cases_Plant_3_Hospital_10 + 249*Cases_Plant_3_Hospital_2 + 65*Cases_Plant_3_Hospital_3 + 195*Cases_Plant_3_Hospital_4 + 104*Cases_Plant_3_Hospital_5 + 114*Cases_Plant_3_Hospital_6 + 51*Cases_Plant_3_Hospital_7 + 335*Cases_Plant_3_Hospital_8 + 508*Cases_Plant_3_Hospital_9 + 124*Cases_Plant_4_Hospital_1 + 199*Cases_Plant_4_Hospital

**9. Solve your optimization model and print its status and the optimal objective function value. NOTE: Your optimal objective function value should be 44930.0**

In [9]:
model.solve(solver)
print(f"Status: {pulp.LpStatus[model.status]}")
print(f"Total cost = ${pulp.value(model.objective)}")

Status: Optimal
Total cost = $44930.0


**10. Output the value of each of your variables at optimality.**

In [10]:
for v in model.variables():
    print(f"{v.name} = {v.varValue}")

Cases_Plant_1_Hospital_1 = 0.0
Cases_Plant_1_Hospital_10 = 0.0
Cases_Plant_1_Hospital_2 = 0.0
Cases_Plant_1_Hospital_3 = 71.0
Cases_Plant_1_Hospital_4 = 0.0
Cases_Plant_1_Hospital_5 = 0.0
Cases_Plant_1_Hospital_6 = 129.0
Cases_Plant_1_Hospital_7 = 0.0
Cases_Plant_1_Hospital_8 = 0.0
Cases_Plant_1_Hospital_9 = 0.0
Cases_Plant_2_Hospital_1 = 0.0
Cases_Plant_2_Hospital_10 = 0.0
Cases_Plant_2_Hospital_2 = 0.0
Cases_Plant_2_Hospital_3 = 0.0
Cases_Plant_2_Hospital_4 = 0.0
Cases_Plant_2_Hospital_5 = 50.0
Cases_Plant_2_Hospital_6 = 0.0
Cases_Plant_2_Hospital_7 = 0.0
Cases_Plant_2_Hospital_8 = 0.0
Cases_Plant_2_Hospital_9 = 0.0
Cases_Plant_3_Hospital_1 = 0.0
Cases_Plant_3_Hospital_10 = 0.0
Cases_Plant_3_Hospital_2 = 0.0
Cases_Plant_3_Hospital_3 = 50.0
Cases_Plant_3_Hospital_4 = 0.0
Cases_Plant_3_Hospital_5 = 0.0
Cases_Plant_3_Hospital_6 = 0.0
Cases_Plant_3_Hospital_7 = 100.0
Cases_Plant_3_Hospital_8 = 0.0
Cases_Plant_3_Hospital_9 = 0.0
Cases_Plant_4_Hospital_1 = 0.0
Cases_Plant_4_Hospital_10 = 0

**11. Use Python to list each Hospital's Demand and total supply of Vaccine sent from your solution.**

HINT:  `model.constraints` returns you a dictionary.   You can inspect the keys of the dictionary with `model.constraints.keys()`.  Each key is the name of a constraint -- the name you provided when you created the constraint.  

So, for example if you used the name `hospital_demand_1` for the supply/demand constraint for hospital 1 then you can retrieve the shortfall, or slack, in the solution with `model.constraints['hospital_demand_1'].slack` 



In [11]:
for hospital in hospitals:
    total_received = 0
    for plant in plants:
        total_received += shipments[(plant['name'], hospital['name'])].varValue
        slack = hospital['demand'] - total_received
    print(f"{hospital['name']} needed {hospital['demand']} vaccine cases and received {total_received}. Slack = {slack}.")

Hospital 1 needed 50 vaccine cases and received 0.0. Slack = 50.0.
Hospital 2 needed 75 vaccine cases and received 40.0. Slack = 35.0.
Hospital 3 needed 121 vaccine cases and received 121.0. Slack = 0.0.
Hospital 4 needed 231 vaccine cases and received 0.0. Slack = 231.0.
Hospital 5 needed 147 vaccine cases and received 50.0. Slack = 97.0.
Hospital 6 needed 190 vaccine cases and received 129.0. Slack = 61.0.
Hospital 7 needed 100 vaccine cases and received 100.0. Slack = 0.0.
Hospital 8 needed 151 vaccine cases and received 51.0. Slack = 100.0.
Hospital 9 needed 49 vaccine cases and received 49.0. Slack = 0.0.
Hospital 10 needed 172 vaccine cases and received 100.0. Slack = 72.0.


**12. DO NOT ANSWER HERE, BUT PREPARE FOR CLASS DISCUSSION**
- How might you consider not only transportation costs, but also take into account a *fair* allocation of vaccine to the hospitals?